In [2]:
import sys
sys.path.append('../')
from rocket_rag.utils import *
from rocket_rag.node_indexing import *
from rocket_rag.vector_store import *

c:\Users\HAOXUAN\miniconda3\envs\agents2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load = '40kg'
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]

In [4]:
# np.random.seed(42)

rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Random inference sample: {if_ts_filename}')
if_rocket_feature = fit_transform([if_ts_filename],
                                    field='current',
                                    smooth=True,
                                    smooth_ws=15,
                                    tolist=False,
                                    verbo=False)
print(f'ROCKET features shape: {if_rocket_feature.shape}')

Random inference sample: ../data/inference/40kg\spalling1\spalling1_40_10_4.csv
ROCKET features shape: (1, 20000)


In [5]:
node_indexer = NodeIndexer()
nodes = node_indexer.load_node_indexing(f'../store/nodes_{load}.pkl')

2024-05-29 17:13:09.363 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-29 17:13:09.775 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.


In [6]:
vector_store = VectorStore()
vector_store.add(nodes)

In [7]:
ids, dists = vector_store.ridge_query(if_rocket_feature)
print(ids)
print(dists)

['spalling1_40_8_5']
1.0


### Get the fault diagnosis reuslt

In [8]:
import json
from prompts import fault_diagnosis_prompt

In [9]:
CONFIG_FILE = "../config/configs.json"
with open(CONFIG_FILE) as f:
    config = json.load(f)
    GOOGLE_API_KEY = config["google_api_key"]
    GOOGLE_CSE_ID = config["google_cse_id"]
    OPENAI_API_KEY = config["openai_api_key"]
    GPT_MODEL = config["gpt_model"]

In [10]:
from openai import OpenAI

In [11]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

LOCAL = False

# Point to the local server or use remote OpenAI GPT API
local_llm_client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed") 
openai_client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', config["openai_api_key"]))
client = local_llm_client if LOCAL else openai_client
model = "local-model" if LOCAL else GPT_MODEL

In [12]:
history = [
    {"role": "system", "content": fault_diagnosis_prompt.sys_prompt},
    {"role": "user", "content": fault_diagnosis_prompt.ridge_prompt.format(res=str(ids), score=dists)},
]

completions = client.chat.completions.create(
    model=model,
    messages=history,
    response_format= { "type": "json_object" },
    temperature=0.1,
    stream=True,
)

new_message = {"role": "assistant", "content": ""}

for chunk in completions:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        new_message["content"] += chunk.choices[0].delta.content

history.append(new_message)
fault_diagnosis_res = new_message['content']

{
    "fault_type": "spalling",
    "degradation_level": 1,
    "retrieval_result(s)": "spalling1_40_8_5",
    "score": 1.0,
    "distances": [],
    "description": "The actuator is experiencing surface damage in the ball-screw mechanism at a low level, affecting its smoothness and efficiency. Immediate attention is recommended to prevent further deterioration and ensure optimal performance."
}

In [13]:
print(type(json.loads(fault_diagnosis_res)))
fault_diagnosis_json = json.loads(fault_diagnosis_res)
fault_type = fault_diagnosis_json['fault_type']
fault_description = fault_diagnosis_json['description']

<class 'dict'>


### Use multi-query generation for query parsing 

In [14]:
from prompts import multi_queries_gen_prompt

In [15]:
mq_messages = [
    {"role": "system", "content": multi_queries_gen_prompt.sys_prompt},
    {"role": "user", "content": multi_queries_gen_prompt.user_prompt.format(fault_type=fault_type, 
                                                                            fault_description=fault_description, 
                                                                            num=str(5))},
]
            
completions = client.chat.completions.create(
    model=model, # this field is currently unused
    messages=mq_messages,
    temperature=0.1,
    tools=[{}],
    stream=True,
)

new_message = {"role": "assistant", "content": ""}

for chunk in completions:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        new_message["content"] += chunk.choices[0].delta.content

history.append(new_message)
multi_queries_gen = new_message['content']

BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].type'.", 'type': 'invalid_request_error', 'param': 'tools[0].type', 'code': 'missing_required_parameter'}}

In [50]:
import re

def formalize_query(query: str):
    """Preprocess the query for the vector store query
    
    Remove some symbols including '-', '"', '.' and indexing numbers or patterns like 1. 2. 3. ...
    """
    query = query.strip().replace('"', '').replace('. ', '')
    pattern = re.compile(r'[-0-9]+|\d+\. ')
    result = pattern.sub('', query)
    return result.strip()

In [51]:
generated_queries = [formalize_query(query) for query in multi_queries_gen.split('\n')]
generated_queries

['repair techniques for spalling in linear actuators',
 'how to fix surface damage in ballscrew mechanisms of linear actuators',
 'maintenance procedures for spalling in linear actuators',
 'preventing further damage from spalling in linear actuators',
 'best practices for addressing spalling in linear actuator systems']

### Use external tools for query search for decision-support
Here use Google chrome web browser for a proof-of-concept validation

In [17]:
import json
from googleapiclient.discovery import build

In [18]:
def call_google(query: str, **kwargs):
    """ Call the google chrome for searching online """
    
    service = build(serviceName="customsearch", 
                    version="v1", 
                    developerKey=GOOGLE_API_KEY,
                    static_discovery=False)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
    res_items = res["items"]
    res_snippets = [r['snippet'] for r in res_items]
    return str(res_snippets)

# A quick validation for the call_google
print(call_google(query=generated_queries[0]))

["This includes reconditioning the actuator's ball screw, repairing and/or replacing worn or damaged internal components, replacing cover bands, and replacing all\xa0...", 'Jan 7, 2010 ... This service bulletin also gives instructions to remove/replace and repair the Horizontal Stabilizer Trim Actuator if a damaged. Ballscrew or\xa0...', 'Often premature flaking or abnormal damage may lead to machine failure. Cause of the problem may include careless handling, excessive misalignment, insufficient\xa0...', 'for Duff-Norton translating ball screw actuators. ... Do not allow actuator travel to go beyond cata- log closed height of actuator or serious damage to internal\xa0...', 'Damage Condition, Possible Causes, Countermeasures. The raceways of the screw shaft and ball nut and/or the surface of the ball peel off like scales because\xa0...', 'Check ball nut threads for damage and replace if necessary. 7. Check retaining wire location. Some are free to rotate to other part of nut when the r

In [24]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "call_google",
            "description": "Call the google chrome web browser to search online based on a given query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query string for searching online",
                    },
                },
                "required": ["query"],
            },
        },
    }
]

In [25]:
available_tools = {"call_google": call_google}

### Use ReAct Prompting to call the external functions

In [ ]:
# from rocket_rag.prompts import react_prompt
# from colorama import Fore, Back, Style

# # regular expression regex patterns
# action_re = re.compile('^Action: (\w+): (.*)$')
# answer_re = re.compile("Answer: ")
# answers = []

# chrome_messages = [
#     {"role": "system", "content": react_prompt.sys_prompt},
#     {"role": "user", "content": react_prompt.user_prompt.format(query=generated_queries[0])},
# ]

# while True:
#     response = client.chat.completions.create(
#         model=model,
#         messages=chrome_messages,
#     )
    
#     # Get the response from the GPT and add it as a part of memory
#     response_msg = response.choices[0].message.content
#     history.append({"role": "assistant", "content": response_msg})
    
#     # If the respionse contains the keyword "Answer: ", then return
#     if answer_re.search(response_msg):
#         print(Fore.YELLOW + response.choices[0].message.content)
#         print(Style.RESET_ALL)
#         answers.append(answer_re.search(response_msg).group(1))
#         break
    
#     # Print the thinking process
#     print(Fore.GREEN + response_msg)
#     print(Style.RESET_ALL)

#     # Take actions
#     actions = [action_re.match(a) for a in response_msg.split("\n") if action_re.match(a)]
#     if actions:
#         action, action_input = actions[0].groups()
#         try:
#             print(Fore.CYAN + f" -- running {action} {action_input}")
#             print(Style.RESET_ALL)
#             # Apply available tools for the function execution
#             obervation = available_tools[action](action_input) 
#             print(Fore.BLUE + f"Observation: {obervation}")
#             print(Style.RESET_ALL)
#             history.append({"role": "user", "content": "Observation: " + obervation})
#         except:
#             raise NotImplementedError

### Use GPT function calling to use external tools

In [32]:
from tqdm.auto import tqdm

answers = []
for i in tqdm(range(len(generated_queries))):
    loguru.logger.debug(f'Processing {i+1}/{len(generated_queries)} query...')

    gpt_tool_call_messages = [
        {"role": "system", "content": "You are a helpful chatbot that can use web browser to offer reliable searching results to human beings."},
        {"role": "user", "content": f"Search for the following query using Google web browser: {generated_queries[i]}"}
    ]

    first_response = client.chat.completions.create(
        model=model,
        messages=gpt_tool_call_messages,
        tools=tools,
    )
    # print(first_response)
    
    gpt_tool_call_messages.append(first_response.choices[0].message)
    tool_calls = first_response.choices[0].message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_tools[function_name]
            function_args = json.loads(tool_call.function.arguments)
            loguru.logger.debug(function_args)
            function_response = function_to_call(**function_args)

            gpt_tool_call_messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
    
    second_response = client.chat.completions.create(
        model=model,
        messages=gpt_tool_call_messages,
        tools=tools,
    )
    # print(second_response)
    
    answers.append(second_response.choices[0].message.content)

  0%|          | 0/5 [00:00<?, ?it/s]2024-03-18 12:33:53.585 | DEBUG    | __main__:<module>:5 - Processing 1/5 query...
2024-03-18 12:33:55.613 | DEBUG    | __main__:<module>:26 - {'query': 'how to repair spalling damage in ballscrew actuators'}
 20%|██        | 1/5 [00:23<01:35, 23.87s/it]2024-03-18 12:34:17.457 | DEBUG    | __main__:<module>:5 - Processing 2/5 query...
2024-03-18 12:34:19.458 | DEBUG    | __main__:<module>:26 - {'query': 'best practices for preventing spalling in linear actuators'}
 40%|████      | 2/5 [00:53<01:21, 27.12s/it]2024-03-18 12:34:46.853 | DEBUG    | __main__:<module>:5 - Processing 3/5 query...
2024-03-18 12:34:49.054 | DEBUG    | __main__:<module>:26 - {'query': 'replacement options for ballscrew actuators with spalling damage'}
 60%|██████    | 3/5 [01:28<01:02, 31.01s/it]2024-03-18 12:35:22.490 | DEBUG    | __main__:<module>:5 - Processing 4/5 query...
2024-03-18 12:35:24.804 | DEBUG    | __main__:<module>:26 - {'query': 'diagnosing spalling in linear

### Gathering all searching results 

In [33]:
import asyncio
import nest_asyncio
from openai import AsyncClient

nest_asyncio.apply()

In [34]:
asyncclient = AsyncClient(api_key=OPENAI_API_KEY)

async def async_combined_responses(query_responses: List[str], sys_prompt: str, num_children=3, debug=False):
    """Async version of combining responses from different nodes"""

    node_batch_prompts = []
    for idx in range(0, len(query_responses), num_children):
        node_batch = query_responses[idx:idx+num_children]
        node_batch_text = "\n\n".join([node for node in node_batch])

        temp_prompt = f""" \
        Context information is below: 
        {node_batch_text}
        Given the context information and not prior knowledge, summarize and present the result with detailed descriptions.
        Combined result: \
        """

        node_batch_prompts.append(temp_prompt)
    
    tasks = [asyncclient.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": tp}
        ]) for tp in node_batch_prompts
    ]

    combined_responses = await asyncio.gather(*tasks)
    new_texts = [r.choices[0].message.content for r in combined_responses]

    if len(new_texts) == 1:
        loguru.logger.info(f"Combined all responses to one. Done")
        return new_texts[0]
    else:
        loguru.logger.info(f"Combined into {len(new_texts)} responses, keep combining")
        if debug:
            loguru.logger.info(new_texts)
        return await async_combined_responses(new_texts, sys_prompt)

In [35]:
combined_result_sys_prompt = f"You are a helpful assistant that can summarize and extract useful information from give text as follow."
combined_result = await async_combined_responses(answers, combined_result_sys_prompt)

2024-03-18 12:37:16.426 | INFO     | __main__:async_combined_responses:35 - Combined into 2 responses, keep combining
2024-03-18 12:38:12.273 | INFO     | __main__:async_combined_responses:32 - Combined all responses to one. Done


### Save all contents into one markdown file

In [ ]:
queries_expr = [str(i+1) + '. ' + generated_queries[i] + "\\" for i in range(len(generated_queries))]
queries_expr = "\n".join(queries_expr)
anwsers_expr = ('\n'.join(answers)).split('\n')

In [36]:
report = "*FAULT DIAGNOSIS REPORT\n\n" + fault_diagnosis_res + "\n\n"\
"Searching for: \n" + queries_expr + "\n\n"
suggestions = "Maintenance suggestions: \n" + combined_result

In [37]:
import datetime

In [38]:
LOG_DIR = '../logs/'

In [39]:
dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
save_filename = 'fault-diagnosis-' + dt + '.md'
save_filepath = os.path.join(LOG_DIR, save_filename)

try:
    with open(save_filepath, 'w', encoding='utf-8') as f:
        f.write(report)
        for i in range(len(anwsers_expr)):
            f.write(f"{anwsers_expr[i]} \\")
        f.write(suggestions)
    loguru.logger.info(f"Saved report to {save_filepath}")
except Exception as e:
    raise Exception(f"Failed to save report to {save_filepath}")

2024-03-18 12:38:27.811 | INFO     | __main__:<module>:8 - Saved report to ../logs/fault-diagnosis2024-03-18-12-38-27.md
